##### 101 Formulaic Alphas
- Based on [101 Formulaic Alphas](https://arxiv.org/pdf/1601.00991.pdf), Zura Kakushadze, arxiv, 2015

##### Imports & Settings

In [1]:
# Parameters
top = 10  # default value

In [2]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.feature_selection \
        import mutual_info_regression
        
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [4]:
idx= pd.IndexSlice
sns.set_style('whitegrid')

##### Loading Data

In [5]:
ohlcv = ['open', 'high', 'low', 'close', 'volume', 'market_cap']

DATA_STORE = Path('/home/sayem/Desktop/Project/data/assets.h5')

lock_path = "/tmp/assets_h5_file.lock"  # Choose a path for the lock file

from filelock import FileLock

top = 500

with FileLock(lock_path):
    with pd.HDFStore(DATA_STORE) as store:
        data = (store[f'data/top{top}_dataset']
                .loc[:, ohlcv + ['ret_01d', 'sector', 'ret_fwd_01d']]
                .rename(columns={'ret_01d': 'returns'})
                .sort_index())


In [6]:
# # Meta Labeling
# data_ohlcv = data[['open', 'high', 'low', 'close', 'volume']].copy()
# T = data['ret_fwd_01d'].std()
# data['primary_target'] = (data['ret_fwd_01d'].abs() > T).astype(int)

# data['secondary_target'] = np.where(data['primary_target'] == 1, 
#                                     np.where(data['ret_fwd_01d'] > 0, 1, -1),
#                                     np.nan)

In [7]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# def compute_targets(data):
#     # Primary model's target
#     threshold = 0.01  # Threshold for returns; for demonstration purposes
#     data['primary_target'] = np.where(data['ret_fwd_01d'].abs() > threshold, 1, 0)

#     # Secondary model's target
#     data['secondary_target'] = np.where(data['ret_fwd_01d'] > 0, 1, -1)
#     data['secondary_target'] = data['secondary_target'] * data['primary_target']

#     return data

# def plot_targets(data, ticker, start_date, end_date):
#     # Filter data for the given ticker and time frame
#     data_filtered = compute_targets(data.loc[ticker].loc[start_date:end_date])

#     # Set up the plot
#     fig, ax1 = plt.subplots(figsize=(15, 6))

#     # Plotting primary target
#     line1, = ax1.step(data_filtered.index, data_filtered['primary_target'], \
#                       label='Primary Target', where='mid', color='blue')

#     # Right axis for secondary target
#     ax2 = ax1.twinx()
#     line2, = ax2.step(data_filtered.index, data_filtered['secondary_target'], \
#                       color='red', label='Secondary Target', where='mid', linestyle='--')
    
#     ax1.set_title(f'Targets for {ticker} ({start_date} to {end_date})', fontsize=16)
#     ax1.set_xlabel('Date', fontsize=14)
    
#     # Left axis for primary target
#     ax1.set_ylim(0, 1)
#     ax1.set_yticks([0, 1])
#     ax1.set_yticklabels(['No Bet', 'Bet'], fontsize=12)

#     # Right axis for secondary target
#     ax2.set_ylim(-1, 1)
#     ax2.set_yticks([-1, 0, 1])
#     ax2.set_yticklabels(['Short', '', 'Long'], fontsize=12)

#     # Adjusting legend
#     lines = [line1, line2]
#     labels = [line.get_label() for line in lines]
#     ax1.legend(lines, labels, loc='upper left')

#     # Pad the spacing between the two y-axes
#     ax2.spines['right'].set_position(('outward', 60))  
#     ax2.yaxis.set_ticks_position('right')
#     ax2.yaxis.set_label_position('right')

#     plt.tight_layout()
#     plt.show()

# # To use the function:
# plot_targets(data, 'AA', '2013-01-01', '2013-01-31')

In [8]:
# Calculate rolling average volumes
data['adv10'] = data.groupby('ticker').rolling(10).volume.mean().reset_index(0, drop=True)
data['adv20'] = data.groupby('ticker').rolling(20).volume.mean().reset_index(0, drop=True)
data['adv50'] = data.groupby('ticker').rolling(50).volume.mean().reset_index(0, drop=True)
data['adv81'] = data.groupby('ticker').rolling(81).volume.mean().reset_index(0, drop=True)
data['adv150'] = data.groupby('ticker').rolling(150).volume.mean().reset_index(0, drop=True)
adv180 = data['adv180'] = data.groupby('ticker').rolling(180).volume.mean().reset_index(0, drop=True)

# Ensure no duplicate indices
data = data[~data.index.duplicated(keep='first')]

# Unstacking once
o = data.open.unstack('ticker')
h = data.high.unstack('ticker')
l = data.low.unstack('ticker')
c = data.close.unstack('ticker')
v = data.volume.unstack('ticker')
r = data.returns.unstack('ticker')
sector = data.sector.unstack('ticker')
market_cap = data.market_cap.unstack('ticker')

# Compute vwap only once
vwap = (o + h + l + c) / 4

# Assemble the datasets dictionary
datasets = {
    'o': o,
    'h': h,
    'l': l,
    'c': c,
    'v': v,
    'r': r,
    'adv10': data['adv10'].unstack('ticker'),
    'adv20': data['adv20'].unstack('ticker'),
    'adv50': data['adv50'].unstack('ticker'),
    'adv81': data['adv81'].unstack('ticker'),
    'adv150': data['adv150'].unstack('ticker'),
    'adv180': data['adv180'].unstack('ticker'),
    'vwap': vwap,
    'sector': sector,
    'market_cap': market_cap
}


In [9]:
import inspect

def apply_alpha_function(alpha_number, datasets):
    """
    Apply the given alpha function (by number) to the datasets.

    Args:
    - alpha_number (int): The number of the alpha function to apply (e.g., 1, 2, 3...)
    - datasets (dict): A dictionary containing the available datasets.

    Returns:
    - The result of the alpha function applied to the datasets.
    """
    # Get the alpha function based on the number
    alpha_func_name = f'alpha{alpha_number:03}'
    if alpha_func_name not in globals():
        raise ValueError(f"No function named {alpha_func_name} found!")
    alpha_func = globals()[alpha_func_name]  # Fetch directly from globals dictionary

    # Get the list of arguments that the function expects
    expected_args = inspect.signature(alpha_func).parameters.keys()

    # Filter the datasets to only include the expected arguments
    filtered_datasets = {k: v for k, v in datasets.items() if k in expected_args}

    # Apply the function using ** to unpack the dictionary into function arguments
    return alpha_func(**filtered_datasets)

In [10]:
import concurrent.futures
import os  
from formulaic_alphas import *
import logging

# Setup logging
logging.basicConfig(filename="alpha_errors.log", level=logging.ERROR)

# [Your dataset definitions here...]

alphas_p = data[['returns', 'ret_fwd_01d']].copy()
error_alphas = []

def apply_alpha_and_store(alpha):
    try:
        result = apply_alpha_function(alpha, datasets)
        return alpha, result, None  # None indicates no error
    except Exception as e:
        logging.error(f"Error applying Alpha {alpha:03}: {str(e)}")
        return alpha, None, str(e)  # Return the error message

success_count = 0

# Number of processes to use for parallelization
num_processes = min(15, os.cpu_count())  # Adjust this as needed

with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
    # Use the executor to map the function over the alphas
    for alpha, result, error in executor.map(apply_alpha_and_store, range(1, 102)):
        if error is None:
            alphas_p[f'alpha_{alpha:03}'] = result
            success_count += 1
            print(f"Alpha {alpha:03} applied successfully and stored in 'alphas_p' DataFrame!")
        else:
            error_alphas.append(alpha)
            print(f"Error applying Alpha {alpha:03}. Check the logs for details.")

# Print the summary
print(f"{success_count} out of 101 alphas have been added to the 'alphas_p' DataFrame.")
if error_alphas:
    print(f"Alphas with errors: {', '.join(map(str, error_alphas))}")


Alpha 001 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 002 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 003 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 004 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 005 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 006 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 007 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 008 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 009 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 010 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 011 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 012 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 013 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 014 applied successfully and stored in 'alphas_p' DataFrame!
Alpha 015 applied successfully and stored in 'alphas_p' DataFr

In [11]:
# import concurrent.futures
# import os  
# from formulaic_alphas import *

# alphas_p = data[['returns', 'ret_fwd_01d']].copy()

# def apply_alpha_and_store(alpha):
#     try:
#         result = apply_alpha_function(alpha, datasets)
#         return alpha, result, None  # None indicates no error
#     except Exception as e:
#         return alpha, None, str(e)  # Return the error message

# success_count = 0

# # Number of processes to use for parallelization
# num_processes = min(15, os.cpu_count())  # Adjust this as needed

# with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
#     # Use the executor to map the function over the alphas
#     for alpha, result, error in executor.map(apply_alpha_and_store, range(1, 102)):
#         if error is None:
#             alphas_p[f'alpha_{alpha:03}'] = result
#             success_count += 1
#             print(f"Alpha {alpha:03} applied successfully and stored in 'alphas_p' DataFrame!")
#         else:
#             print(f"Error applying Alpha {alpha:03}: {error}")

# # Print the summary
# print(f"{success_count} out of 101 alphas have been added to the 'alphas_p' DataFrame.")

In [12]:
import pandas as pd
import gc
from utils import optimize_dataframe

df_optimized = optimize_dataframe(alphas_p.copy())

Data memory before optimization: 901.43 MB
Data memory after optimization: 376.09 MB
Reduced by: 58.28%


In [13]:
with pd.HDFStore(DATA_STORE) as store:
    store.put(f'factor/top{top}_dataset_alpha101', \
        df_optimized, format='table', data_columns=True)

In [ ]:
from utils import clear_large_vars
clear_large_vars(threshold_size_in_MB=100)